# Retrieval Application

## Vector Database (Vector DB)
Resources
- [How-to guides](https://python.langchain.com/v0.2/docs/how_to/#vector-stores)
  - [Vectorstores](https://python.langchain.com/v0.2/docs/integrations/vectorstores/): A vector store that stores embedded data and performs similarity search.
    1. [Elasticsearch](https://python.langchain.com/v0.2/docs/integrations/vectorstores/elasticsearch/)
    2. [Milvus](https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/)
    3. [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Environment Setup

Sources  
- [langchain-chroma](https://pypi.org/project/langchain-chroma/)

In [1]:
from importlib.metadata import version
# !pip install langchain
# Select langchain to 0.1.20
try:
    print('langchain package version',version('langchain'))
    assert version('langchain') == '0.1.20'
except:
    !pip install langchain==0.1.20

# !pip install --upgrade langchain
# print('langchain package version',version('langchain'))

langchain package version 0.1.20


In [2]:
#!pip install -qU langchain-huggingface
# Select langchain-huggingface to 0.0.3
try:
    print('langchain-huggingface package version',version('langchain-huggingface'))
    assert version('langchain-huggingface') =='0.0.3'#'0.2.11'
except:
    !pip install langchain-huggingface==0.0.3
    #0.2.11 (if any)

# !pip install -qU langchain-huggingface
# print('langchain-huggingface package version',version('langchain-huggingface'))

langchain-huggingface package version 0.0.3


In [3]:
# Select langchain-chroma to 0.1.3
try:
    print('langchain_chroma package version',version('langchain_chroma'))
    assert version('langchain_chroma') == '0.1.3'
except:
    !pip install langchain_chroma==0.1.3

# !pip install -qU langchain_chroma==0.1.3
# print('langchain_chroma package version',version('langchain_chroma'))

langchain_chroma package version 0.1.3


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import os
#check current directory
os.getcwd()

'/content'

In [5]:
# check folders / files in current directory
!dir

chroma	Data  sample_data


In [6]:
import langchain_chroma
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Import Data
- Data source: [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings)

**Features of Use**
- id:
  1. job_id  

- Main data  

  **Texts**  
  1. _description_
  1. _skills_desc_  

  **Vector**  
  1. title
  2. _description_
  3. _skills_desc_  
  
- Meta data:  
  1. title
  1. location
  1. min_salary
  1. pay_period
  1. job_posting_url
  
  


In [7]:
df = pd.read_csv('./Data/archive/postings.csv', delimiter=',') #, engine="python"

print(f"The variables (features) of the data:\n{df.columns}")
df.head(5)

The variables (features) of the data:
Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [8]:
df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,3906267117,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,195000.0,YEARLY,"Walnut Creek, CA",56120.0,1.0,NaN,...,NaN,1.713571e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,94595.0,6013.0
123845,3906267126,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,NaN,United States,1124131.0,3.0,NaN,...,NaN,1.713572e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN
123846,3906267131,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,NaN,"Spokane, WA",90552133.0,3.0,NaN,...,NaN,1.713572e+12,epsoperations.bamboohr.com,0,FULL_TIME,NaN,NaN,NaN,99201.0,53063.0
123847,3906267195,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,NaN,"Texas, United States",2793699.0,4.0,NaN,...,NaN,1.713573e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN


In [9]:
# df['job_post'] = df['description'].astype('str') + df['skills_desc'].astype('str')
# df[['description', 'skills_desc', 'job_post']].head()

In [10]:
df['job_post'] = np.where(df['skills_desc'].isna(), df['description'].astype('str'), df['skills_desc'].astype('str'))
df[['description', 'skills_desc', 'job_post']].head()

,description,skills_desc,job_post
0,Job descriptionA leading real estate firm in N...,Requirements: \n\nWe are seeking a College or ...,Requirements: \n\nWe are seeking a College or ...
1,"At Aspen Therapy and Wellness , we are committ...",NaN,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...,We are currently accepting resumes for FOH - A...,We are currently accepting resumes for FOH - A...
3,Senior Associate Attorney - Elder Law / Trusts...,This position requires a baseline understandin...,This position requires a baseline understandin...
4,Looking for HVAC service tech with experience ...,NaN,Looking for HVAC service tech with experience ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

## Select Data
**Purpose**:
- Insert data samples into Vector Database (VectorDB), working as knowledge base

**Note**:
- For demonstration, I pick 50% of data related to a specific job tiltes, and 50% other random job titles

In [12]:
# define yourself

## job title you're interested
keywords = ['data science', 'data scientist', 'data analyst']
# keywords = ['data analytics', 'data scientist', 'hr analytics', 'hr data analyst']

## Number of relevant / irrelevant samples
N = 300

In [13]:
condition = df['title'].str.contains('|'.join(keywords), case=False, na=False)
df_ds = df[condition]
# 'case=False' makes the search case-insensitive
# 'na=False' ensures that NaN values are not considered in the search

N_ds = min(len(df_ds), N)
df_ds = df_ds.sample(n=N_ds)
print(f"There're {N_ds} samples searched according to keywords.")
df_ds.head(10)

There're 300 samples searched according to keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
58443,3901950460,Emeritus,Data Analyst,Emeritus is committed to teaching the skills o...,100000.0,YEARLY,United States,10367223.0,9.0,NaN,...,1.713475e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,90000.0,NaN,NaN,Emeritus is committed to teaching the skills o...
57390,3901943542,Bernhard,Senior Data Analyst,About Us\nBernhard's core mission is deliverin...,NaN,NaN,"Metairie, LA",15211162.0,2.0,NaN,...,1.713471e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,70001.0,22051.0,About Us\nBernhard's core mission is deliverin...
1277,3884434978,Outlier,"Senior Data Analyst - AI Training (Remote, Con...",Job Type: Contract\n\nWe don’t sponsor work vi...,NaN,HOURLY,United States,92583550.0,258.0,55.0,...,1.712350e+12,boards.greenhouse.io,0,CONTRACT,USD,BASE_SALARY,114400.0,NaN,NaN,Job Type: Contract\n\nWe don’t sponsor work vi...
27471,3891275011,Data Glacier,Data Science intern -DIN18,The ideal candidate will use their passion for...,NaN,NaN,United States,69739391.0,9.0,NaN,...,1.712891e+12,NaN,0,INTERNSHIP,NaN,NaN,NaN,NaN,NaN,The ideal candidate will use their passion for...
19090,3888961649,Eversource Energy,"Data Analyst, Energy Efficiency (Hybrid)",\n\n\nThis data analyst position is responsibl...,96000.0,YEARLY,"Berlin, CT",10051.0,4.0,NaN,...,1.712635e+12,jobs.eversource.com,0,FULL_TIME,USD,BASE_SALARY,91000.0,6037.0,9003.0,\n\n\nThis data analyst position is responsibl...
98937,3904989573,Blue Yonder,Sr. Staff Data Scientist,Role : Sr Staff Data Scientist - Technical Lea...,189615.0,YEARLY,"Dallas, TX",2716085.0,9.0,NaN,...,1.713458e+12,jda.wd5.myworkdayjobs.com,0,FULL_TIME,USD,BASE_SALARY,170000.0,75201.0,48113.0,Role : Sr Staff Data Scientist - Technical Lea...
27191,3891267987,Experis,Data Analyst,Job Title: Data Analyst \n\n Work Schedule: ...,NaN,HOURLY,"New Albany, OH",2203697.0,2.0,42.0,...,1.712892e+12,click.appcast.io,0,CONTRACT,USD,BASE_SALARY,87360.0,43054.0,39049.0,Job Title: Data Analyst \n\n Work Schedule: ...
70868,3902844857,PriceSenz,Health Data Scientist,Overview: The NIEHS is seeking a skilled indiv...,NaN,NaN,"Durham, NC",9328925.0,4.0,NaN,...,1.713563e+12,NaN,0,CONTRACT,NaN,NaN,NaN,27701.0,37063.0,Overview: The NIEHS is seeking a skilled indiv...
5668,3885102764,ATC,Data Analyst-VA,Job Title: Entry Level Business Analyst / Prod...,NaN,NaN,"Virginia, United States",2736308.0,2.0,NaN,...,1.712403e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,Job Title: Entry Level Business Analyst / Prod...
63089,3902344156,ATC,Data Analyst-VA,Job Title: Entry Level Business Analyst / Prod...,NaN,NaN,"Virginia, United States",2736308.0,2.0,NaN,...,1.713528e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,Job Title: Entry Level Business Analyst / Prod...


In [14]:
df_others = df[~condition].sample(n=N)
print(f"There're {len(df_others)} samples searched not contained in keywords.")
df_others.head(10)

There're 300 samples searched not contained in keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
49611,3901357379,Arrowmac,Communications Specialist,Are you ready to take your communication skill...,31.00,HOURLY,"Illinois, United States",1851642.0,215.0,NaN,...,1.713453e+12,NaN,0,CONTRACT,USD,BASE_SALARY,61360.0,NaN,NaN,Are you ready to take your communication skill...
109941,3905368113,Bloomberg,E-Trading STP Implementation Specialist EQ API...,Bloomberg is a global leader in business and f...,175000.00,YEARLY,"New York, NY",2494.0,5.0,NaN,...,1.713493e+12,click.appcast.io,0,FULL_TIME,USD,BASE_SALARY,147500.0,10001.0,36061.0,Bloomberg is a global leader in business and f...
2348,3884447181,Vendition,Sales Development Representative,Vendition is seeking high-performing SDRs to j...,80000.00,YEARLY,New York City Metropolitan Area,18723495.0,104.0,NaN,...,1.712351e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,NaN,NaN,Vendition is seeking high-performing SDRs to j...
74384,3902960533,Helios HR,Agreement Manager-National Democratic Institute,"Our client, the National Democratic Institute,...",110200.00,YEARLY,"Washington, DC",51863.0,29.0,NaN,...,1.713281e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,99200.0,20001.0,11001.0,"Our client, the National Democratic Institute,..."
87227,3904388599,Overlake Medical Center & Clinics,Surgical Technologist (0.6 FTE / Days),Welcome to a medical center where you're the c...,50.87,HOURLY,"Bellevue, WA",26356.0,4.0,NaN,...,1.713394e+12,overlakemedicalcenter.wd5.myworkdayjobs.com,0,PART_TIME,USD,BASE_SALARY,85560.8,98004.0,53033.0,Welcome to a medical center where you're the c...
23031,3889745341,Allied Universal,Surveillance Investigator,Allied Universal® Compliance and Investigation...,NaN,NaN,"Billings, MT",10696913.0,6.0,NaN,...,1.712667e+12,jobs.aus.com,0,FULL_TIME,NaN,NaN,NaN,59101.0,30111.0,Allied Universal® Compliance and Investigation...
101188,3905243144,The Sixth Floor Museum at Dealey Plaza,Chief Experience Officer,"Chief Experience OfficerOur client, The Sixth ...",NaN,NaN,"Dallas, TX",109298.0,7.0,NaN,...,1.713465e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,75201.0,48113.0,"Chief Experience OfficerOur client, The Sixth ..."
50918,3901377438,Progress,"Senior Manager, Customer Success",We’re Progress – we offer the best products to...,NaN,NaN,United States,3496.0,27.0,NaN,...,1.713459e+12,app.jobvite.com,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,We’re Progress – we offer the best products to...
118448,3906095241,Brightview Senior Living,Chef,About Us\n\nWe are honored that Brightview Sen...,NaN,NaN,"Bethesda, MD",433995.0,3.0,NaN,...,1.713553e+12,careers.brightviewseniorliving.com,0,FULL_TIME,NaN,NaN,NaN,20814.0,24031.0,About Us\n\nWe are honored that Brightview Sen...
123280,3906258097,National Staffing Solutions,Licensed Practical Nurse,"At National Staffing Solutions, we have the pr...",NaN,NaN,"Danvers, MA",1211063.0,4.0,NaN,...,1.713567e+12,www.nationalstaff.com,0,FULL_TIME,NaN,NaN,NaN,1923.0,25009.0,"At National Staffing Solutions, we have the pr..."


In [15]:
df_select = pd.concat([df_ds, df_others])

print(f"There're totally {len(df_select)} samples for inserting VectorDB.")

There're totally 600 samples for inserting VectorDB.


# Create VectorDB

## Create Indexer: Construct the Container (aka. collection) for VectorDB

In [16]:
encoder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and

In [17]:
encoder

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))



In [19]:
if not persistent_client.list_collections():
    print('empty')

In [20]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))

if not persistent_client.list_collections():
    print(f"{collection_name} not in collection (DB) yet!")
else:
    if collection_name in persistent_client.list_collections()[0].name:
        print(f"{collection_name} is already in collection and would be deleted!")
        persistent_client.delete_collection(collection_name)

print(f"Create collection: {collection_name}!")
vector_store = Chroma(
    client=persistent_client,
    collection_name=collection_name,
    embedding_function=encoder,
    persist_directory="./chroma_langchain_db",  #save data locally, remove if not neccesary
)

#print("check all functions/attribute for Chroma:\n", dir(vector_store))

# if collection_name in vector_store.list_collections():
#     vector_store.delete_collection(collection_name)


collection_postings is already in collection and would be deleted!
Create collection: collection_postings!


## Indexing: Insert Data into VectorDB


In [21]:
from langchain_core.documents import Document
ids = []
documents = []
for index, row in df_select.iterrows():
    id_current = str(index)
    ids.append(id_current)

    min_salary = row['min_salary']
    if pd.isna(min_salary):
        min_salary = 0  # or any default value you prefer

    document_current = Document(
        page_content=row['job_post'],
        metadata={"title": row['title'],
                  "location": row['location'],
                  "min_salary": min_salary,
                  "pay_period": row['pay_period'],
                  "job_posting_url": row['job_posting_url'],
                 },
        id=row['job_id'],
    )
    documents.append(document_current)
print(f"There are {len(documents)}.\n")
print(f'Example document content:\n{document_current}')

There are 600.

Example document content:
page_content='\nLiberty Cares\nWith Compassion\nAt Liberty Hospice we understand the unique needs of our patients and families facing terminal illness. That is why Liberty Hospice provides our hospice patients with state-of-the-art care and pain management services, delivered by our specially trained staff with emphasis on strength, dignity and compassion.\nWe are currently seeking an experienced:\n\nHOSPICE CHAPLAIN\n\nFull Time\n\n(Covering Halifax and surrounding counties) \nJob Description: Provides spiritual support to patients/caregivers. Develops and maintains a resource group of clergy. Acts as a liaison between hospice and community. Documents records of visits to patients/caregivers. Works collaboratively as a member of the Interdisciplinary Team. Other job duties as assigned.\n\nJob Requirements: Bachelor’s degree from an accredited college/university in religion, human services, theology, philosophy and/or be an ordained minister of

In [24]:
import time
start = time.time()
vector_store.add_documents(documents=documents, ids=ids)
end = time.time()
print('Time spent (min.) for data insertion: \t', (end-start)/60)

Time spent (min.) for data insertion: 	 15.704426503181457


## Search: VectorDB as Retriever

### Vector Search (Similarity Search)
According to the query, find k=5 best possible job posts

In [25]:
query = "I am a passionate job seeker with a strong desire to embark on a career in data science. Having recently graduated with a degree in Computer Science, I have honed my skills in Python programming and developed a deep interest in machine learning. During my studies, I immersed myself in various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. My journey into data science has been driven by a fascination with uncovering hidden patterns in data and using these insights to solve real-world problems. I am now seeking an entry-level data scientist role where I can leverage my Python expertise and enthusiasm for machine learning to contribute to a dynamic team, learn from experienced professionals, and continue to grow my skills in this exciting field."
print(query)

I am a passionate job seeker with a strong desire to embark on a career in data science. Having recently graduated with a degree in Computer Science, I have honed my skills in Python programming and developed a deep interest in machine learning. During my studies, I immersed myself in various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. My journey into data science has been driven by a fascination with uncovering hidden patterns in data and using these insights to solve real-world problems. I am now seeking an entry-level data scientist role where I can leverage my Python expertise and enthusiasm for machine learning to contribute to a dynamic team, learn from experienced professionals, and continue to grow my skills in this exciting field.


In [26]:
results = vector_store.similarity_search_with_score(
    query , k=10,
)
i =1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.471218] Junior Data Scientist - Python /Modeling
---------------------
           Junior Data Scientist – ( Python / Modeling and Operat ions ) 

 Expert Technical Solutions has an immediate opening for a Junior Data Scientist with an industry leading client in Atlanta , GA. Our client is looking for a highly motivated and research oriented Data Scientist who will play a vital role in enhancing our cli ent ’ s contact center solutions through the development of advanced NLP algorithms and models. You will work in a collaborative team environment to explore and create solutions to industry problems using machine learning and state-of-the-art large language models. You bring to this team environment a strong foundation in machine learning and deep learning, as well as the curiosity and grit to see a project to delivery. 

T his is a permanent , HYBRID (2 days onsite, Monday and Tuesday every week) opportunity offering st rong pay, ex cellent growth opportunities, and outstand

### Vector Search with Filtering 1
According to the query, find k=5 best possible job posts which have minimum salary greater than (gt) 100000

In [27]:
results = vector_store.similarity_search_with_score(
    query , k=3, filter={"min_salary": {"$gt": 100000}}
)# perator: $gt, $gte, $lt, $lte, $ne, $eq, $in, $nin
i = 1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.540949] Senior Data Scientist
---------------------
           Senior Data ScientistThe ideal candidate's favorite words are learning, data, scale, and agility. You will leverage your strong collaboration skills and ability to extract valuable insights from highly complex data sets to ask the right questions and find the right answers.  ResponsibilitiesCollaborate with subject matter and technical experts to strategize analyses, utilize existing algorithms/tools, and derive actionable insights.Recommend data framework or architecture to optimize data analytics.Use storyboards, wireframes, mockups, and simple prototypes; and develop them into real solutions.Help to develop design specifications and proof-of-concept solutions in response to business insight needs.Ensure that outputs, including data visualization and associated messaging, are of consistent high quality and ready for client presentation.Collect feedback from the application of models, contributing to model refi

# Retrieval and Generation

**Vector Database (Vector DB)**
Resources
- [How-to guides](https://python.langchain.com/v0.2/docs/how_to/#vector-stores)
  - [Vectorstores](https://python.langchain.com/v0.2/docs/integrations/vectorstores/): A vector store that stores embedded data and performs similarity search.
    1. [Elasticsearch](https://python.langchain.com/v0.2/docs/integrations/vectorstores/elasticsearch/)
    2. [Milvus](https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/)
    3. [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Preface
## Environment Setup

Sources  
- [langchain-chroma](https://pypi.org/project/langchain-chroma/)

In [28]:
# OpenAI
# Update OpenAI to 1.42.0
try:
    print('openai package version',version('openai'))
    assert version('openai') == '1.42.0'
except:
    !pip install openai==1.42.0

openai package version 1.42.0


In [29]:
!pip install tiktoken

# Connect to VectorDB & LLM Agent
## Connect to VectorDB (Chroma)

In [30]:
import chromadb
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint

collection_name = "collection_postings"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
persistent_client = chromadb.PersistentClient()
print(persistent_client.list_collections())

vector_store = Chroma(client=persistent_client,
                      collection_name=collection_name,
                      embedding_function=embeddings)

# try:
#   if collection_name in persistent_client.list_collections()[0].name:
#       print(f"Collection '{collection_name}' exists!")
#       # Get the existing collection
#       # vector_store = persistent_client.get_collection(collection_name)
#       vector_store = Chroma(client=persistent_client,
#                             collection_name=collection_name,
#                             embedding_function=embeddings)
# except:
#     print(f"Collection '{collection_name}' does not exist!")

[Collection(id=9b70e796-c6b4-45b5-a720-624de2ce8339, name=collection_postings)]


In [31]:
# prompt: how can I see the data in vector_store?

# Get all the documents in the vector store
documents = vector_store.get()

# Print the documents
print(documents)

# Alternatively, you can get the embeddings and ids
embeddings = vector_store.get()['embeddings']
ids = vector_store.get()['ids']

# Print the embeddings
print(embeddings)

# Print the ids
ids

{'ids': ['58443', '57390', '1277', '27471', '19090', '98937', '27191', '70868', '5668', '63089', '81765', '83144', '98770', '108747', '84504', '42920', '107168', '101483', '5399', '41998', '63368', '51643', '48647', '29957', '3730', '114452', '49237', '118279', '74873', '115052', '52419', '46950', '29113', '111194', '65102', '21215', '28444', '100783', '99084', '84694', '102035', '44016', '70770', '38342', '122066', '6455', '86214', '97143', '100616', '103113', '63690', '75642', '24892', '108466', '90109', '117455', '52380', '9360', '51662', '51372', '38826', '67390', '99384', '74521', '38692', '51901', '12953', '6356', '98017', '67677', '21015', '51895', '30222', '59038', '57086', '12413', '114153', '96427', '22531', '67493', '77914', '91802', '70326', '80140', '121441', '37684', '65353', '31328', '57092', '41064', '50526', '21150', '23827', '95843', '76497', '366', '41024', '17846', '116610', '101835', '35632', '29166', '13074', '92604', '112825', '6777', '98453', '61753', '104602', 

['58443',
 '57390',
 '1277',
 '27471',
 '19090',
 '98937',
 '27191',
 '70868',
 '5668',
 '63089',
 '81765',
 '83144',
 '98770',
 '108747',
 '84504',
 '42920',
 '107168',
 '101483',
 '5399',
 '41998',
 '63368',
 '51643',
 '48647',
 '29957',
 '3730',
 '114452',
 '49237',
 '118279',
 '74873',
 '115052',
 '52419',
 '46950',
 '29113',
 '111194',
 '65102',
 '21215',
 '28444',
 '100783',
 '99084',
 '84694',
 '102035',
 '44016',
 '70770',
 '38342',
 '122066',
 '6455',
 '86214',
 '97143',
 '100616',
 '103113',
 '63690',
 '75642',
 '24892',
 '108466',
 '90109',
 '117455',
 '52380',
 '9360',
 '51662',
 '51372',
 '38826',
 '67390',
 '99384',
 '74521',
 '38692',
 '51901',
 '12953',
 '6356',
 '98017',
 '67677',
 '21015',
 '51895',
 '30222',
 '59038',
 '57086',
 '12413',
 '114153',
 '96427',
 '22531',
 '67493',
 '77914',
 '91802',
 '70326',
 '80140',
 '121441',
 '37684',
 '65353',
 '31328',
 '57092',
 '41064',
 '50526',
 '21150',
 '23827',
 '95843',
 '76497',
 '366',
 '41024',
 '17846',
 '116610',
 '

## Connect to Agent (Call OpenAI API)

In [32]:
import openai
from google.colab import userdata
#initiate the OpenAI client using the API key
# openai_api_key = os.environ["OPENAI_API_KEY"]
openai_api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)
client

# Retrieval and Generation Application

## Prepare Prompt

In [33]:
extraction_prompt = ''' You are a carear consuler who helps job seekers to find their dream jobs, you give professional advice tailored to the need of your client (i.e., job seeker) according to the following information:
    1. Query: Your client's question (enclosed in <query> tag below) that you need to answer
    2. Specification: The job post information (enclosed in <specification> tag below) that might best meets your client's requirements

Upon receiving your aforementioned information, you need to proceed with the following precedures:
Step 1. Analyze your client's abilities, including hard and soft skills.
Step 2. Analyze and summarize the skills needed for the best possible jobs in the job specification
Step 3. Summarize your client's strengths that are already sufficient for the job application.
Step 4. Summarize your client's weaknesses that they need to improve in order to meet the job requirements.
Step 5. Finally, give them advice how to get the jobs mentioned in job specification according the reasoning above.

Question:
    <query>{query}</query>
Job Post Information:
    <specification>{specification}</specification>
Advice:
'''

## Preprare Input Query

In [34]:
query = "I recently graduated with a Bachelor degree in Computer Science, I use Python and have good grades in machine learning and deep learning. I had various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. I am now seeking an entry-level data scientist or data analyst role."

## Search Results based on Query

In [35]:
results = vector_store.similarity_search_with_score(
    query , k=5, #filter={"title": {"$in": keywords}}
)
i=0
specification = ""
for res, score in results:
    print(f"[{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    specification += ('Title: ' + res.metadata['title'] +'\n ' + res.page_content)
    i+=1

[0][SIM=0.569052] Data Scientist
---------------------
           Job title: Data ScientistJob Location: Bentonville, AR - OnsiteJob Type: Fulltime
Job Description:Proven experience in deploying real-time AI/ML models using Google Cloud Platform.Strong programming skills in Python and PySpark.Proficiency with SQL and relational databases, data warehouses, and Big Query.Experience in scaling marketing-related AI/ML solutions such as cross/upsell, recommended systems, and category propensity.Experience in deploying and managing Large scale Machine Learning Models is a plusExpertise with classical ML algorithm like K-NN, LSH, logistic regression, linear regression, SVM, Random forest and clustering.Good understanding of ML & DL algorithms and frameworks (Scikit-learn,Spacy, Tensor flow/Keras/ PyTorch)Experience in deep learning Algorithms like MLP, CNN, RNN, LSTMs and GANs, Transformers and LLMs.Excellent programming skills in PythonExpertise in Google Cloud and operationalization of mode

In [36]:
print(specification)

Title: Data Scientist
 Job title: Data ScientistJob Location: Bentonville, AR - OnsiteJob Type: Fulltime
Job Description:Proven experience in deploying real-time AI/ML models using Google Cloud Platform.Strong programming skills in Python and PySpark.Proficiency with SQL and relational databases, data warehouses, and Big Query.Experience in scaling marketing-related AI/ML solutions such as cross/upsell, recommended systems, and category propensity.Experience in deploying and managing Large scale Machine Learning Models is a plusExpertise with classical ML algorithm like K-NN, LSH, logistic regression, linear regression, SVM, Random forest and clustering.Good understanding of ML & DL algorithms and frameworks (Scikit-learn,Spacy, Tensor flow/Keras/ PyTorch)Experience in deep learning Algorithms like MLP, CNN, RNN, LSTMs and GANs, Transformers and LLMs.Excellent programming skills in PythonExpertise in Google Cloud and operationalization of models using MLOPs.Experience in scheduling job

## Get Final Response

In [37]:
prompt_all = extraction_prompt.format(query=query, specification=specification)
print(prompt_all)

 You are a carear consuler who helps job seekers to find their dream jobs, you give professional advice tailored to the need of your client (i.e., job seeker) according to the following information:
    1. Query: Your client's question (enclosed in <query> tag below) that you need to answer
    2. Specification: The job post information (enclosed in <specification> tag below) that might best meets your client's requirements

Upon receiving your aforementioned information, you need to proceed with the following precedures:
Step 1. Analyze your client's abilities, including hard and soft skills.
Step 2. Analyze and summarize the skills needed for the best possible jobs in the job specification
Step 3. Summarize your client's strengths that are already sufficient for the job application.
Step 4. Summarize your client's weaknesses that they need to improve in order to meet the job requirements.
Step 5. Finally, give them advice how to get the jobs mentioned in job specification according t

In [38]:
import tiktoken

# Define a function to count tokens for a given prompt and model
def count_tokens(text, model="gpt-3.5-turbo-instruct"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Count the number of tokens in the prompt
prompt_tokens = count_tokens(prompt_all);print(f"total prompt tokens = {prompt_tokens}")

# Token limit for gpt-3.5-turbo-instruct
token_limit = 4097

# Ensure the total tokens (prompt + response) is within the limit
# Assume you want the model to generate a maximum of 1000 tokens in the response
response_max_tokens = 1000
if prompt_tokens + response_max_tokens > token_limit:
    print('total token size exceeds limit, start trimming!')
    # Calculate the allowable prompt length
    max_prompt_tokens = token_limit - response_max_tokens

    # Trim the prompt to fit within the token limit
    trimmed_prompt = prompt_all[:max_prompt_tokens]

    # Notify user about trimming
    print(f"Prompt trimmed from {prompt_tokens} to {max_prompt_tokens} tokens.")
    print("final prompt_all:\n",prompt_all)

    # Update the prompt with the trimmed version
    prompt_all = trimmed_prompt
else:
    print('total token size doesn\'t  exceeds limit, good job!')



total prompt tokens = 2018
total token size doesn't  exceeds limit, good job!


In [39]:
response = client.completions.create(model="gpt-3.5-turbo-instruct",
                                     prompt=prompt_all,
                                     max_tokens=response_max_tokens)
print(response.choices[0].text)

Dear [Client],

Congratulations on your recent graduation with a Bachelor degree in Computer Science! Your skills in Python, machine learning, and deep learning make you a strong candidate for an entry-level data scientist or data analyst role.

After analyzing your abilities and reviewing the job specifications, I believe that your strengths align well with the requirements for the Data Scientist and Senior Data Scientist positions. Your experience with building predictive models and analyzing large datasets, as well as your proficiency with Python and SQL, make you a strong fit for these roles. However, there are some areas where you can continue to improve in order to increase your chances of getting hired.

First, it would be beneficial for you to gain more experience with deploying real-time AI/ML models using Google Cloud Platform. This is a specific skill mentioned in both job descriptions, and having more experience with it will make you a stronger candidate. Additionally, gain

# What If: Generation without Application

In [40]:
extraction_prompt = ''' You are a carear consuler who helps job seekers to find their dream jobs, you give professional advice tailored to the need of your client (i.e., job seeker) according to the following information:
    1. Query: Your client's question (enclosed in <query> tag below) that you need to answer


Upon receiving your aforementioned information, you need to proceed with the following precedures:
Step 1. Analyze your client's abilities, including hard and soft skills.
Step 2. Analyze and summarize the skills needed for the best possible jobs
Step 3. Summarize your client's strengths that are already sufficient for the job application.
Step 4. Summarize your client's weaknesses that they need to improve in order to meet the job requirements.
Step 5. Finally, give them advice how to get the jobs.

Question:
    <query>{query}</query>

Advice:
'''

prompt_all = extraction_prompt.format(query=query)

In [41]:
response = client.completions.create(model="gpt-3.5-turbo-instruct",
                                     prompt=prompt_all,
                                     max_tokens=response_max_tokens)
print(response.choices[0].text)

Step 1. Analyze your client's abilities, including hard and soft skills.
    Based on the information provided, your client has a strong foundation in computer science, with a focus on python and machine learning/deep learning. They also have experience with real-world projects that demonstrate their expertise in these areas.

Step 2. Analyze and summarize the skills needed for the best possible jobs
    For an entry-level data scientist or data analyst role, the most important skills are typically technical proficiency in programming languages such as Python, R, and SQL, as well as experience with machine learning and data analysis. Additionally, soft skills such as communication, problem-solving, and critical thinking are highly valued in these roles.

Step 3. Summarize your client's strengths that are already sufficient for the job application.
    Your client's strengths lie in their technical proficiency in Python and their experience with machine learning and data analysis. They 